# ChiffonCore bring-up (Ultra96V2 / PYNQ)

1. Load bitstream (Overlay)
2. Write `.hex` to DRAM (CMA buffer)
3. Set `dram_base` / `entry_pc` and `START` signal via regbus


In [ ]:
from pathlib import Path
import numpy as np
from pynq import Overlay, allocate


## 1) Load Overlay


In [ ]:
BIT_PATH = "design_1.bit"
ol = Overlay(BIT_PATH)
ol


## 2) Get regbus handle


In [ ]:
# Find regbus
cands = [k for k in ol.ip_dict.keys() if "regbus" in k.lower()]
print("regbus candidates:", cands)
assert len(cands) > 0, "regbus IP not found. HINT: check ol.ip_dict."
regbus = getattr(ol, cands[0])
regbus


## 3) Define bootctrl

- `STATUS`  : `BOOT_BASE + 0x0`  (bit1=run, bit0=hold_reset)
- `CTRL`    : `BOOT_BASE + 0x4`  (bit0=hold_reset level, bit1=START W1P)
- `DRAMBASE`: `BOOT_BASE + 0x8`
- `ENTRYPC` : `BOOT_BASE + 0xC`


In [ ]:
BOOT_BASE = 0x1000

RA_STATUS   = BOOT_BASE + 0x0000
RA_CTRL     = BOOT_BASE + 0x0004
RA_DRAMBASE = BOOT_BASE + 0x0008
RA_ENTRYPC  = BOOT_BASE + 0x000C


## 4) Load .hex and write DDR buffer

32bit per line

- word0 -> `[31:0]`
- word1 -> `[63:32]`


In [ ]:
HEX_PATH = "program.hex"
BYTESWAP32 = False

def parse_hex_words(path: str):
    words = []
    for line in Path(path).read_text().splitlines():
        line = line.strip()
        if not line or line.startswith("#") or line.startswith("//"):
            continue
        # allow 0x prefix
        if line.startswith("0x") or line.startswith("0X"):
            line = line[2:]
        v = int(line, 16) & 0xFFFF_FFFF
        if BYTESWAP32:
            v = int.from_bytes(v.to_bytes(4, "big"), "little")
        words.append(v)
    return words

words32 = parse_hex_words(HEX_PATH)
print("32-bit words:", len(words32))

# pack 2x32 -> 1x64 (pure Python shift to avoid numpy left_shift dtype issues)
n_qwords = (len(words32) + 1) // 2
buf = allocate(shape=(n_qwords,), dtype=np.uint64, cacheable=False)

for qi in range(n_qwords):
    lo = words32[2*qi] if (2*qi) < len(words32) else 0
    hi = words32[2*qi + 1] if (2*qi + 1) < len(words32) else 0
    # little-endian packing: word0 -> [31:0], word1 -> [63:32]
    buf[qi] = np.uint64((hi << 32) | lo)

buf.flush()

entry_pc = int(buf.device_address)  # DDR physical address
print(f"DDR buffer device_address = 0x{entry_pc:08x}")
assert entry_pc <= 0xFFFF_FFFF, "TODO: AXI_ADDR_WIDTH=32 なので 4GB を超えるアドレスは使えない"


## 5) Write settings to bootctrl and START

`CTRL = 0b10` を 1回書くと：
- hold_reset = 0（解除）
- run = 1（START W1P）


In [ ]:
regbus.write(RA_CTRL, 0x1)  # hold_reset=1

# base and entry
regbus.write(RA_DRAMBASE, 0x0000_0000)
regbus.write(RA_ENTRYPC,  entry_pc & 0xFFFF_FFFF)

# START (release reset + run)
regbus.write(RA_CTRL, 0x2)

st = regbus.read(RA_STATUS)
print(f"STATUS=0x{st:08x}  (run={(st>>1)&1}, hold_reset={st&1})")


## 6) TODO: display?
